## Requeriments

In [ ]:
# Instal boto3 
%pip install boto3

# Install dotenv
%pip install python-dotenv

In [41]:
# Reload notebook to have saved changes of others modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Download information from AWS S3

In [42]:
# Import libraries
from dotenv import load_dotenv
from download_s3 import get_file_folders
from download_s3 import download_files
import boto3
import os
load_dotenv()

# Initiate env variables
ACCESS_KEY = os.getenv('ACCESS_KEY')
SECRET_KEY = os.getenv('SECRET_KEY')
BUCKET_NAME = os.getenv('BUCKET_NAME')
BUCKET_PREFIX = os.getenv('BUCKET_PREFIX')
LOCATION_DIR= os.getenv('LOCATION_DIR') 

# Let's use Amazon S3, connect to AWS S3.
s3_client = boto3.client(
    "s3",
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY
    )

In [43]:
# Get folders and files names
folders, files = get_file_folders(s3_client,
                                  BUCKET_NAME,
                                  BUCKET_PREFIX
)

# Download folders and files from S3 to local
download_files(s3_client=s3_client,
               bucket_name=BUCKET_NAME,
               local_path=LOCATION_DIR,
               prefix=BUCKET_PREFIX,
               folders=folders,
               file_names=files
)

"wget http://trax-geometry.s3.amazonaws.com/cvpr_challenge/SKU110K_fixed.tar.gz"

## Evaluation EDA
##### Imagenes
- Evaluar todas las imágenes con el fin de encontrar imágenes corruptas o con formato no valido.

##### Imagenes & csv
- Validar imagenes que no tengan bounding boxes con referencia con csv.
- Evaluar que todas las imágenes tienen información en el csv.

##### csv
- Validad que no existan valores nulos.
- Validar valores enteros en coordendas.
- Validar que sean positivos.
- Validar que los xs y ys no sean iguales.
- Validar si el bounding cubre un producto con outliers
- Validar superposicieon de bounding boxes

In [45]:
# Import libraries
import pandas as pd
import numpy as np

In [33]:
# Directions
txt_file = "/Users/fabioalvarez/Documents/1. Anyone AI/Final Project/retail_prediction/data/annotations/readme.txt"
test_file = "/Users/fabioalvarez/Documents/1. Anyone AI/Final Project/retail_prediction/data/annotations/annotations_test.csv"
train_file = "/Users/fabioalvarez/Documents/1. Anyone AI/Final Project/retail_prediction/data/annotations/annotations_train.csv"
val_file = "/Users/fabioalvarez/Documents/1. Anyone AI/Final Project/retail_prediction/data/annotations/annotations_val.csv"

In [98]:
# Read txt file
with open(txt_file) as f:
    txt = f.read()

# Extract column names
column_names = txt.split(":")[1].strip()
columns = column_names.split(',')

# Read csv and check null values and add column to identify dataset
test_df = pd.read_csv(test_file, index_col=False, names=columns)
test_df['set'] = 'test'

# Read csv and check null values and add column to identify dataset
train_df = pd.read_csv(train_file, index_col=False, names=columns)
train_df['set'] = 'train'

# Read csv and check null values and add column to identify dataset
val_df = pd.read_csv(val_file, index_col=False, names=columns)
val_df['set'] = 'val'

# Unificate datasets
df = pd.concat([test_df, train_df, val_df], axis=0, ignore_index=True)

In [101]:
# Check data info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1730996 entries, 0 to 1730995
Data columns (total 9 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   image_name    object
 1   x1            int64 
 2   y1            int64 
 3   x2            int64 
 4   y2            int64 
 5   class         object
 6   image_width   int64 
 7   image_height  int64 
 8   set           object
dtypes: int64(6), object(3)
memory usage: 118.9+ MB


In [103]:
# Check stats about the info
df.describe().round(2)

,x1,y1,x2,y2,image_width,image_height
count,1730996.00,1730996.00,1730996.00,1730996.00,1730996.00,1730996.00
mean,1173.73,1531.98,1294.15,1721.80,2452.50,3220.00
std,709.15,824.72,711.42,813.46,425.52,677.16
min,0.00,0.00,21.00,21.00,480.00,640.00
25%,595.00,898.00,716.00,1098.00,2336.00,2560.00
50%,1153.00,1498.00,1270.00,1675.00,2448.00,3264.00
75%,1702.00,2106.00,1820.00,2286.00,2448.00,3264.00
max,4129.00,4730.00,4323.00,5168.00,4320.00,5312.00


In [104]:
# Check null values in dataframe
df.isnull().value_counts()

image_name  x1     y1     x2     y2     class  image_width  image_height  set  
False       False  False  False  False  False  False        False         False    1730996
dtype: int64

In [93]:
# Check for negative values
numeric_df = df.drop(['image_name', 'class', 'set'], axis=1)

negatives = numeric_df.apply(lambda x : x < 0)
numeric_df[negatives.any(1)]

,x1,y1,x2,y2,image_width,image_height


In [122]:
df.apply(lambda x: x.x1 == x.x2 == x.y1 == x.y2, axis = 1).value_counts()

False    1730996
dtype: int64